# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [10]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print('total rows from CSV source files: {}'.format(len(full_data_rows_list)))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
print('\nsample:\n{}'.format(full_data_rows_list[0:1]))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


total rows from CSV source files: 8056

sample:
[['Rokia TraorÃ\x83Â©', 'Logged In', 'Stefany', 'F', '0', 'White', '274.88608', 'free', 'Lubbock, TX', 'PUT', 'NextSong', '1.54071E+12', '693', 'Zen', '200', '1.54251E+12', '83']]


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('rows inserted into new CSV file: {}'.format(sum(1 for line in f)))

rows inserted into new CSV file: 6821


---
# Part II. Complete the Apache Cassandra coding portion of your project. 

### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [12]:
# make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [13]:
# create a keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)


#### Set Keyspace

In [14]:
# set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

##### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


##### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

##### 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1
##### Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

In [21]:
# create table

query = "CREATE TABLE IF NOT EXISTS song_in_session "
query = query + "(session_id INT, item_in_session INT, artist_name VARCHAR, song_title VARCHAR, song_duration DECIMAL, \
PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)               

In [24]:
# insert data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_in_session (session_id, item_in_session, artist_name, song_title, song_duration)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [34]:
# verify the data was entered into the table

query = "SELECT * FROM song_in_session WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_duration)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Query 2
##### Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [27]:
# create table

query = "CREATE TABLE IF NOT EXISTS song_in_user_session "
query = query + "(user_id INT, session_id INT, artist_name VARCHAR, song_title VARCHAR, user_first_name VARCHAR, \
user_last_name VARCHAR, PRIMARY KEY (user_id, session_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)               

In [29]:
# insert data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_in_user_session (user_id, session_id, artist_name, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4]))

In [33]:
# verify the data was entered into the table

query = "SELECT * FROM song_in_user_session WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.user_first_name, row.user_last_name)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3
##### Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [31]:
# create table

query = "CREATE TABLE IF NOT EXISTS user_song_listen "
query = query + "(user_id INT, user_first_name VARCHAR, user_last_name VARCHAR, song_title VARCHAR, \
PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)               

In [32]:
# insert data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_listen (user_id, user_first_name, user_last_name, song_title)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

In [36]:
# verify the data was entered into the table

query = "SELECT * FROM user_song_listen WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [37]:
query1 = "drop table song_in_session"
query2 = "drop table song_in_user_session"
query3 = "drop table user_song_listen"

try:
    rows1 = session.execute(query1)
    rows2 = session.execute(query2)
    rows3 = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()